## Modeling 

In [72]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import scipy.stats as stats


In [73]:
df = pd.read_csv(r'C:\Users\tonym\Documents\Flatiron\phase_2\Project2\notebooks\Tony\cleanhousedata.csv')

In [74]:
df.head()

,price,view,sqft_living,sqft_lot,bedrooms,bathrooms
0,221900.0,4,1180,5650,2,2
1,538000.0,4,2570,7242,2,7
2,180000.0,4,770,10000,1,2
3,604000.0,4,1960,5000,3,10
4,510000.0,4,1680,8080,2,6


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21010 entries, 0 to 21009
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   price        21010 non-null  float64
 1   view         21010 non-null  int64  
 2   sqft_living  21010 non-null  int64  
 3   sqft_lot     21010 non-null  int64  
 4   bedrooms     21010 non-null  int64  
 5   bathrooms    21010 non-null  int64  
dtypes: float64(1), int64(5)
memory usage: 985.0 KB


In [76]:
df.corr().abs()['price'].sort_values(ascending=False)

price          1.000000
sqft_living    0.682794
bathrooms      0.507509
bedrooms       0.319988
view           0.297368
sqft_lot       0.126884
Name: price, dtype: float64

### Begin to split into train and test 

In [82]:
X = df.drop(columns=['price', 'sqft_lot'])
y = df['price']

In [83]:
# Train / Test Split 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [79]:
# Make a train df so we can see X versus y in our train data
train_df = pd.concat([X_train, y_train], axis=1)

In [64]:
print("X train:")
display(X_train.describe())
print("X Test:")
display(X_test.describe())

X train:


,view,sqft_living,sqft_lot,bedrooms,bathrooms
count,15757.000000,15757.000000,15757.000000,15757.000000,15757.000000
mean,3.739608,2047.495082,10412.887669,2.368027,6.397220
std,0.899162,857.471015,11800.634470,0.895704,2.989242
min,0.000000,370.000000,520.000000,0.000000,0.000000
25%,4.000000,1420.000000,5000.000000,2.000000,4.000000
50%,4.000000,1900.000000,7500.000000,2.000000,7.000000
75%,4.000000,2520.000000,10300.000000,3.000000,8.000000
max,4.000000,7880.000000,104544.000000,8.000000,24.000000


X Test:


,view,sqft_living,sqft_lot,bedrooms,bathrooms
count,5253.000000,5253.000000,5253.000000,5253.000000,5253.000000
mean,3.721302,2051.367028,10392.027032,2.354464,6.400343
std,0.929952,882.123575,11598.095736,0.895745,3.024056
min,0.000000,380.000000,600.000000,0.000000,0.000000
25%,4.000000,1410.000000,5089.000000,2.000000,4.000000
50%,4.000000,1890.000000,7620.000000,2.000000,7.000000
75%,4.000000,2520.000000,10280.000000,3.000000,8.000000
max,4.000000,7480.000000,105415.000000,8.000000,26.000000
